# Exercises Editor
Author: Brent Artuch <br>
Date: 2023-06-07

## Import From SQL

## Introduction

In this notebook we explore how exercise data is logged for a given day of training and calculations to
determine the output of each exercise and what the desired output for the following week will be. The
data entries are recorded at the gym using a simple log app that I developed for Android. The key data
in this notebook will be the `weight` and `reps` of each exercise.

## Step: 1 Imports
### Import packages

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px

### Load the dataset
This dataset is pulled from a SQLite database named `exercises` with four tables: `leg_day`, `chest_day`,
`back_day`, and `shoulder_day`. We will load the desired day using a simple query.

In [2]:
sql_connection = sqlite3.connect('exercises.sqlite')
# Import desired table and set the index.
current_exercises = pd.read_sql('SELECT * FROM leg_day', sql_connection, index_col='index')
current_exercises.tail(10)

,id,date,name,weight,reps
index,,,,,
38,1000039,2023-06-02 00:00:00,Leg Curls,120,20
39,1000040,2023-06-02 00:00:00,Adductor,260,18
40,1000041,2023-06-09 00:00:00,Single Leg Calf Press,280,11
41,1000042,2023-06-09 00:00:00,Standing Calf Press,430,11
42,1000043,2023-06-09 00:00:00,Smith Squat,290,11
43,1000044,2023-06-09 00:00:00,Oblique Crunch Machine,170,12
44,1000045,2023-06-09 00:00:00,Smith Straight Leg Deadlift,290,11
45,1000046,2023-06-09 00:00:00,Leg Extensions,115,12
46,1000047,2023-06-09 00:00:00,Leg Curls,135,13


## Step 2: Data Exploration
Next we will get a sense of the size and contents of `current_exercises`.

In [3]:
current_exercises.shape

(48, 5)

In [4]:
current_exercises.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 0 to 47
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      48 non-null     int64 
 1   date    48 non-null     object
 2   name    48 non-null     object
 3   weight  48 non-null     object
 4   reps    48 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.2+ KB


In [5]:
current_exercises['name'].value_counts()

Single Leg Calf Press          6
Standing Calf Press            6
Smith Squat                    6
Oblique Crunch Machine         6
Smith Straight Leg Deadlift    6
Leg Extensions                 6
Leg Curls                      6
Adductor                       6
Name: name, dtype: int64

## Step 3: Update the Dataset
First we will get each of the unique names from `current_exercises`.

In [6]:
# Set names to a list of unique exercise names
names = current_exercises['name'].unique()
names

array(['Single Leg Calf Press', 'Standing Calf Press', 'Smith Squat',
       'Oblique Crunch Machine', 'Smith Straight Leg Deadlift',
       'Leg Extensions', 'Leg Curls', 'Adductor'], dtype=object)

Next we set the date that we wish to update.

In [7]:
day_of_month = input("Enter the day of the month: DD")
date = f"2023-06-{day_of_month} 00:00:00"
date

'2023-06-16 00:00:00'

### Execute the entry insertion algorithm

In [8]:
for i in range(0, len(names)):

  """ This algorithm sets the elements of a new entry with user input and then inserts it into the
  database table for each name. Id is calculated by last index."""

  weight = input(f"Enter Weight for {names[i]}: ")
  reps = input(f"Enter Reps for {names[i]}: ")
  e_id = current_exercises.iloc[-1, 0] + 1
  new_entry = [e_id, date, names[i], weight, reps]
  current_exercises.loc[len(current_exercises)] = new_entry

# Format the string dates to datetime.
current_exercises['date'] = pd.to_datetime(current_exercises['date'])
current_exercises.tail(len(names))

,id,date,name,weight,reps
index,,,,,
48,1000049,2023-06-16,Single Leg Calf Press,295,8
49,1000050,2023-06-16,Standing Calf Press,340,18
50,1000051,2023-06-16,Smith Squat,340,10
51,1000052,2023-06-16,Oblique Crunch Machine,170,13
52,1000053,2023-06-16,Smith Straight Leg Deadlift,340,9
53,1000054,2023-06-16,Leg Extensions,130,12
54,1000055,2023-06-16,Leg Curls,135,13
55,1000056,2023-06-16,Adductor,260,18


### Data Manipulation
This section exists for convenience if quick changes need to be made to the entries.

In [ ]:
# Update a single item by id and column
# current_exercises.loc[39, 'weight'] = 290
# Drop last row
# current_exercises.drop(index=current_exercises.index[-1], axis=0, inplace=True)
current_exercises.tail(10)

### Update SQL
Now that we have updated our table, we will export the new dataset to SQL. As a precaution, I always
back up the new dataset to CSV through the SQL workbench in the event that an older version is required
in the future.

In [9]:
# Export new chart to database with a date stamp
current_exercises.to_sql('leg_day', sql_connection, if_exists='replace')

# TODO: MANUALLY REVIEW AND EXPORT NEW TABLE TO CSV AS BACKUP!!

56

## Step 5: Calculations
Since I rotate my weights and reps each weak based on performance, we will use the volume equation divided by
the square root of the product of `weight` and `reps` to get the normalized volume output.

$$ Volume = \frac{weight \cdot reps \cdot$ sets}{ \sqrt{weight \cdot reps}} $$

For our purposes the sets variable will always be 1 because only one working set is performed.

In [10]:
current_exercises['volume'] = (current_exercises['weight'].astype(int) * current_exercises['reps'].astype(int)) / np.sqrt(current_exercises['weight'].astype(int) * current_exercises['reps'].astype(int))
current_exercises.head()

,id,date,name,weight,reps,volume
index,,,,,,
0,1000001,2023-04-14,Single Leg Calf Press,250,8,44.721360
1,1000002,2023-04-14,Standing Calf Press,475,10,68.920244
2,1000003,2023-04-14,Smith Squat,380,12,67.527772
3,1000004,2023-04-14,Oblique Crunch Machine,170,20,58.309519
4,1000005,2023-04-14,Smith Straight Leg Deadlift,330,8,51.380930


##### Divide the exercises into two groups `low_volume` and `high_volume`.

## Step 6: Visualization
Next, plot the volume trend for the exercise.

In [12]:
fig = px.line(
  current_exercises, x='date', y='volume', color='name', markers=True,
  height=600,width=800,
  labels={
    'volume': "Weight x Reps",
    'date': "Workout Date",
    'name': "Exercise"
  },
  title="Exercise Volume (Normalized)"
)
fig.update_traces(textposition='top center')
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()

## Leg Day Notes:
* 2023-05-19: Changed Smith Squat range of motion to full so should be marked drop in weight.
* 2023-06-02: Changed Leg Extensions from two legs to one which will account for the weight change.